### [How to stream runnables](https://python.langchain.com/docs/how_to/streaming/)

In [2]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. LLMs and Chat Models

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')

In [5]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| color| of| the| sky| appears| primarily| blue| during| the| day| due| to| a| phenomenon| called| Ray|leigh| scattering|.| This| occurs| when| sunlight| interacts| with| the| Earth's| atmosphere|,| scattering| shorter| blue| wavelengths| of| light| more| than| the| longer| red| wavelengths|.| At| sunrise| and| sunset|,| the| sky| can| display| a| range| of| colors|,| including| reds|,| oranges|,| and| pink|s|,| due| to| the| angle| of| the| sun| and| increased| atmospheric| particles|.| On| cloudy| or| over|cast| days|,| the| sky| may| appear| gray|.||

In [6]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| sky| typically| appears| blue| during| the| day| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| However|,| it| can| change| colors| at| dawn| and| dusk|,| displaying| hues| of| orange|,| pink|,| and| purple|.| At| night|,| the| sky| is| usually| dark|,| but| it| can| also| be| filled| with| stars| and| the| moon|,| creating| a| beautiful| contrast| against| the| darkness|.| Weather| conditions|,| such| as| clouds| and| storms|,| can| also| affect| the| color| of| the| sky|.||

In [7]:
chunks[0]

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-65cfd560-5b1c-48fe-8755-e4e59e1a5deb')

In [8]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='The sky typically appears', additional_kwargs={}, response_metadata={}, id='run-65cfd560-5b1c-48fe-8755-e4e59e1a5deb')

#### B. Chains

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser


async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|uns|aturated|!| 🦜|☔|️||

### C. Working with input Streams

In [11]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467

In [16]:
from langchain_core.output_parsers import JsonOutputParser

# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names

chain  = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [17]:
from langchain_core.output_parsers import JsonOutputParser

#! Funcion generadora, una funcion que usa yield
async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)

chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

#### D. Non-streaming components

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]